In [19]:
import pandas as pd
import matplotlib
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn import tree
df = pd.read_csv('./data-3-28-18-restricted.csv')

# Continuous
df=df.replace("not specified",np.nan)
df['mileage']=df['mileage'].convert_objects(convert_numeric=True)
df['year']=df['year'].convert_objects(convert_numeric=True)
df['price']=df['price'].convert_objects(convert_numeric=True)

#Numerical-normalize the values,change year, multiply mileage by 1000 if less then 1000
df['mileage'] = preprocessing.normalize(df['mileage'].fillna((df['mileage'].mean())).reshape(1,-1), norm='l2').reshape(-1,1)
df['year'] =  df['year'].fillna((df['year'].mean()))
df['year']=df['year']-1990
df['year'] =  preprocessing.normalize(df['year'].fillna((df['year'].mean())).reshape(1,-1)).reshape(-1,1)
df['mileage'] = df['mileage'].fillna((df['mileage'].mean()))
df['mileage']=df['mileage'].apply(lambda x: x if x>1000 else x*1000)
df['price'] = df['price'].fillna((df['price'].mean()))


# Categorical
df['make'] = df['make'].fillna(df['make'].value_counts().index[0])
df['model'] = df['model'].fillna(df['model'].value_counts().index[0])
df['state'] = df['state'].fillna(df['state'].value_counts().index[0])
df['transmission'] = df['transmission'].fillna(df['transmission'].value_counts().index[0])




C:\Users\Rachael\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  
C:\Users\Rachael\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  from ipykernel import kernelapp as app
C:\Users\Rachael\Anaconda2\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  app.launch_new_instance()
C:\Users\Rachael\Anaconda2\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\Rachael\Anaconda2\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: reshape is deprecated and will ra

In [20]:
#df=preprocessing.normalize(df)
features = df[['make', 'model', 'mileage', 'state', 'transmission', 'year']]
labels = df[['price']]

features_encoded = pd.get_dummies(features, columns=['make', 'model', 'state', 'transmission'])
X_train, X_test, Y_train, Y_test = train_test_split(features_encoded, labels, test_size=0.2, train_size=0.8)

# knn

Normalizing the mileage and years took the score from .27 to .78

In [21]:
knn_model = neighbors.KNeighborsRegressor(n_neighbors=10)
knn_model.fit(X_train, Y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=10, p=2,
          weights='uniform')

In [22]:
knn_model.score(X_test, Y_test)

0.78279669599662038

# Decision Tree

In [15]:
dt_model = tree.DecisionTreeRegressor()
dt_model.fit(X_train, Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [16]:
dt_model.score(X_test, Y_test)

0.80460250053360693

# Gradient Boosting

learning rate changed the score from .66 to .83, changing the depth and n_estimators to .86

In [4]:
gb_model = ensemble.GradientBoostingRegressor(learning_rate=.6, n_estimators=200, max_depth=4)
gb_model.fit(X_train, Y_train)

C:\Users\Rachael\Anaconda2\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.6, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [5]:
gb_model.score(X_test, Y_test)

0.86612321419687466